In [9]:
pip install causalinference

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 960.0 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from functools import reduce

# xgb boost
# import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# causal forest 
from econml.dml import CausalForestDML
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor

# sci-kit generalized random forests
from sklearn.model_selection import train_test_split
from skgrf.ensemble import GRFForestRegressor
from skgrf.ensemble import GRFForestCausalRegressor


In [3]:
df = pd.read_csv("imputed_sample.csv")

In [4]:
df.head()

,person_id,household_id,ohp_all_ever_matchn_30sep2009,happiness_12m,health_gen_bin_12m,health_chg_bin_12m,baddays_phys_12m,baddays_ment_12m,health_work_12m,dep_interest_12m,...,phqtot_inp,rx_num_mod_0m,snap_tot_hh_30sep2009,snap_tot_hh_firstn_survey12m,snap_tot_hh_prenotify07,snap_tot_hh_presurvey12m,treatment,usual_care_0m,wave_survey0m,weight_total_inp
0,1.0,100001.0,0.0,2.0,0.0,0.0,15.0,30.0,0.0,3.0,...,5.8,0.2,746.0,2624.0,7453.0,5575.0,1.0,2.4,5.0,0.891361
1,2.0,100002.0,1.0,1.0,0.0,0.0,3.0,12.0,1.0,2.0,...,9.4,1.2,3962.0,5154.0,2404.0,1212.0,1.0,2.6,5.0,1.115141
2,5.0,100005.0,0.0,1.0,0.0,0.0,30.0,30.0,0.0,1.0,...,1.0,1.8,0.0,0.0,0.0,0.0,1.0,2.0,6.0,1.150416
3,6.0,100006.0,0.0,1.0,0.0,0.0,0.0,4.0,0.0,0.0,...,9.0,0.0,3014.0,3176.0,2367.0,2205.0,1.0,4.0,2.0,1.421699
4,8.0,102094.0,0.0,0.0,0.4,0.0,2.0,0.0,0.0,1.0,...,9.0,2.0,3363.0,4439.0,4089.0,3013.0,0.0,1.0,2.0,0.897460


In [5]:
# Filter the data for the variables of interest n
variables_of_interest = [
    'person_id', 'household_id', 'treatment', 'applied_app', 'approved_app', 'ohp_all_ever_matchn_30sep2009',
    'numhh_list', 'wave_survey12m', 'female_list', 'birth_year', 'english_list', 'happiness_12m', 
    'health_gen_bin_12m', 'health_chg_bin_12m', 'baddays_phys_12m', 'baddays_ment_12m', 'health_work_12m', 
    'dep_interest_12m', 'dep_sad_12m', 'cost_any_oop_12m', 'cost_tot_oop_12m', 'cost_any_owe_12m', 
    'cost_tot_owe_12m', 'cost_borrow_12m', 'cost_refused_12m'
]

# Ensure all variables are in the dataset, filter out those that are not
variables_available = [var for var in variables_of_interest if var in df.columns]

# Filter the dataset for the variables of interest that are available
filtered_data = df[variables_available]

# Display the first few rows of the filtered dataset and the list of variables that were not found in the dataset
variables_not_found = set(variables_of_interest) - set(variables_available)
filtered_data.head(), variables_not_found


(   person_id  household_id  treatment  applied_app  \
 0        1.0      100001.0        1.0          1.0   
 1        2.0      100002.0        1.0          0.0   
 2        5.0      100005.0        1.0          0.0   
 3        6.0      100006.0        1.0          1.0   
 4        8.0      102094.0        0.0          0.8   
 
    ohp_all_ever_matchn_30sep2009  numhh_list  happiness_12m  \
 0                            0.0         1.0            2.0   
 1                            1.0         1.0            1.0   
 2                            0.0         1.0            1.0   
 3                            0.0         1.0            1.0   
 4                            0.0         2.0            0.0   
 
    health_gen_bin_12m  health_chg_bin_12m  baddays_phys_12m  baddays_ment_12m  \
 0                 0.0                 0.0              15.0              30.0   
 1                 0.0                 0.0               3.0              12.0   
 2                 0.0              

In [8]:
# Define the treatment, outcome, and covariates
treatment = 'ohp_all_ever_matchn_30sep2009'
outcome = 'health_gen_bin_12m'
covariates = [col for col in filtered_data.columns if col not in [treatment, outcome, 'person_id', 'household_id']]

# Prepare the dataset for analysis
X = filtered_data[covariates]  # Handling missing values by simple imputation (filling with 0)
T = filtered_data[treatment]  # Treatment
Y = filtered_data[outcome] # Outcome, handling missing by mean imputation

# Standardize the covariates to improve model performance
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets for validation
X_train, X_test, T_train, T_test, Y_train, Y_test = train_test_split(X_scaled, T, Y, test_size=0.2, random_state=42)

# Initialize and train the Causal Forest model
causal_forest = CausalForestDML(model_y=GradientBoostingRegressor(),
                                model_t=GradientBoostingRegressor(),
                                discrete_treatment=True,
                                random_state=123)

causal_forest.fit(Y_train, T_train, X=X_train)

# Estimate the causal effect on the test set
causal_effects = causal_forest.effect(X_test)

# Compute average treatment effect on the test data
ate_test = causal_effects.mean()

ate_test

First stage model has discrete target but model is not a classifier!
First stage model has discrete target but model is not a classifier!


-0.0118919307287695

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from causalinference import CausalModel

logit = LogisticRegression(solver='liblinear')
logit.fit(X_scaled, T)

propensity_scores = logit.predict_proba(X_scaled)[:, 1]

auc_score = roc_auc_score(T, propensity_scores)
print(f"AUC score for the propensity model: {auc_score:.4f}")
causal = CausalModel(Y, T, propensity_scores)

causal.est_via_matching(matches=1, bias_adj=True)

print(causal.summary_stats)
print(causal.estimates)

AUC score for the propensity model: 0.7748


KeyError: "None of [Index([4819], dtype='int64')] are in the [index]"

In [11]:
logit_model = LogisticRegression()
logit_model.fit(X_scaled, T)

# propensity scores
propensity_scores = logit_model.predict_proba(X_scaled)[:, 1]

def perform_matching(treatment, control, propensity_scores, caliper=0.05):
    matches = []
    for t_index in treatment:
        # Find the control unit with the closest propensity score within the caliper
        closest = None
        min_distance = caliper
        for c_index in control:
            distance = abs(propensity_scores[t_index] - propensity_scores[c_index])
            if distance < min_distance:
                closest = c_index
                min_distance = distance
        
        if closest is not None:
            matches.append((t_index, closest))
            # Remove the matched control unit to ensure it's not used again
            control.remove(closest)
    
    return matches

# Split treatment and control groups based on treatment variable
treatment_indices = list(T[T == 1].index)
control_indices = list(T[T == 0].index)

# Perform matching
matched_pairs = perform_matching(treatment_indices, control_indices, propensity_scores)

# Convert matched pairs to DataFrame for easier analysis
matched_df = pd.DataFrame(matched_pairs, columns=['Treatment_Index', 'Control_Index'])

print("Number of matched pairs:", len(matched_df))


Number of matched pairs: 5570


In [12]:
matched_df.head()

,Treatment_Index,Control_Index
0,1,22561
1,7,6446
2,8,3789
3,9,10352
4,15,1826
